# Structuring Your Project

**It's Important.** Just as Code Style, API Design, and Automation are essential for a healthy development cycle, Repository structure is a crucial part of your project's structure.

[This is what kenneth reitz recommend](http://docs.python-guide.org/en/latest/).

## Structure of the Repository

- README.rst
- LICENSE
- setup.py
- requirements.txt
- sample/
    - sample/\_\_init__.py
    - sample/core.py
    - sample/helpers.py
- docs/
    - docs/conf.py
    - docs/index.rst
- tests/
    - tests/test_basic.py
    - tests/test_advanced.py

### Specifics

| Parts | Location | Purpose | 
| ----- | ---------| --------|
|The Actual Module| ./sample or ./sample.py | The code of interest. |
|License| ./LICENSE| Lawyering up. |
|Setup.py | ./setup.py | Package and distribution management. |
|Requirements File | ./requirements.txt | Development dependencies.|
|Documentation | ./docs/| Package reference documentation.|
|Test Suite | ./test_sample.py or ./tests | Package integration and unit tests.|
|Makefile | ./Makefile | Generic management tasks.|
|Reaeme| ./README.md(.rst) | The introduction of the ropository.| 


##  Structure of Code

Some signs of a poorly structured project include:

**Multiple and messy circular dependencies**: if your classes Table and Chair in furn.py need to import Carpenter from workers.py to answer a question such as table.isdoneby(), and if conversely the class Carpenter needs to import Table and Chair, to answer the question carpenter.whatdo(), then you have a circular dependency. In this case you will have to resort to fragile hacks such as using import statements inside methods or functions.


## Modules

### Naming restrictions

For example, a layer of a project can handle interfacing with user actions, while another would handle low-level manipulation of data. 

The most natural way to separate these into two layers:
1. all interfacing functionality in one file
2. all low-level operations in another file. 

In this case, the interface file needs to `import` the low-level file. This is done with the import and from ... import statements.

As soon as you use import statements you use modules. These can be either:
- Built-in modules such as os and sys. 
- Third-party modules you have installed in your environment.
- Or your project’s internal modules.

To keep in line with the style guide: 
1. Keep module names **short, lowercase, and there is no need to separate words**.

2. Don’t namespace with underscores, use submodules instead.

3. Be sure to avoid using special symbols like the dot (.) or question mark (?)**. (In the case of my.spam.py Python expects to find a spam.py file in a folder named my which is not the case.)

4. **Using other characters (spaces or hyphens) in module names will prevent importing (- is the subtract operator)**.

```python
# OK
import library.plugin.foo
# not OK
import library.foo_plugin
# wrong
import library.plugin-foo
import library.plugin foo
import library.plugin?foo
```
### Import mechanism

Aside from some naming restrictions, nothing special is required for a Python file to be a module, but you need to understand the import mechanism in order to use this concept properly and avoid some issues.

#### Step 1 : Search the module in path
Concretely, the import modu statement will look for the proper file, which is modu.py in: 
1. First, the same directory as the caller if it exists. 
2. If it is not found, the Python interpreter will search for modu.py in the “path” recursively and raise an ImportError exception if it is not found.

#### Step 2 : Execute the module in an  isolated scope
Once modu.py is found, the Python interpreter will execute the module in an isolated scope. Any top-level statement in modu.py will be executed, including other imports if any. Function and class definitions are stored in the module’s dictionary.

#### Step 3 : Available to the caller in module's namespace
Then, the module’s variables, functions, and classes will be available to the caller through the module’s namespace, a central concept in programming that is particularly helpful and powerful in Python.

#### Note
```
In many languages, an include file directive is used by the preprocessor to take all code found in the file and ‘copy’ it into the caller’s code. 

It is different in Python: the included code is isolated in a module namespace, which means that you generally don’t have to worry that the included code could have unwanted effects, e.g. override an existing function with the same name.
```

### from ... import ... statement

It is possible to simulate the more standard behavior by using a special syntax of the import statement: `rom modu import *`. This is generally considered bad practice. Using import * makes code harder to read and makes dependencies less compartmentalized.

Using `from modu import func` is a way to pinpoint the function you want to import and put it in the global namespace. While much less harmful than import * because it shows explicitly what is imported in the global namespace, its only advantage over a simpler import modu is that it will save a little typing.

#### Very Bad
```python
[...]
from modu import *
[...]
x = sqrt(4)  # Is sqrt part of modu? A builtin? Defined above?
```
#### Better 
```python
from modu import sqrt
[...]
x = sqrt(4)  # sqrt may be part of modu, if not redefined in between
```
#### Best
```python
import modu
[...]
x = modu.sqrt(4)  # sqrt is visibly part of modu's namespace
```


## Packages

Python provides a very straightforward packaging system, which is simply an extension of the module mechanism to a directory.

### Package mechanism

Any directory with an __init__.py file is considered a Python package. The different modules in the package are imported in a similar manner as plain modules, but with a special behavior for the __init__.py file, which is used to gather all package-wide definitions.

```    
sound/                          Top-level package
      __init__.py               Initialize the sound package
      formats/                  Subpackage for file format conversions
              __init__.py
              wavread.py
              wavwrite.py
              aiffread.py
              aiffwrite.py
              auread.py
              auwrite.py
              ...
      effects/                  Subpackage for sound effects
              __init__.py
              echo.py
              surround.py
              reverse.py
              ...
      filters/                  Subpackage for filters
              __init__.py
              equalizer.py
              vocoder.py
              karaoke.py
              ...  
```

A file modu.py in the directory pack/ is imported with the statement import pack.modu. This statement will:
1. look for an __init__.py file in pack, execute all of its top-level statements. 
2. Then look for a file named pack/modu.py and execute all of its top-level statements.
3. After these operations, any variable, function, or class defined in modu.py is available in the pack.modu namespace.

#### Note
```
In Python 3, using absolute path in import statement, while relative path is used in Python 2.
```
### \_\_init\_\_.py

A commonly seen issue is to add too much code to `__init__.py` files. When the project complexity grows, there may be sub-packages and sub-sub-packages in a deep directory structure. In this case, importing a single item from a sub-sub-package will require executing all `__init__.py` files met while traversing the tree.

Leaving an `__init__.py` file empty is considered normal and even a good practice, if the package’s modules and sub-packages do not need to share any code.

### import very.deep.module as mod
Lastly, a convenient syntax is available for importing deeply nested packages: import very.deep.module as mod. This allows you to use mod in place of the verbose repetition of very.deep.module.

###  Intra-package References

- you can use **absolute imports** to refer to submodules of siblings packages.

For example, if the `module sound.filters.vocoder` needs to use the `echo module in the sound.effects package`, it can use 
```python
from sound.effects import echo
```

- You can also write relative imports, with the from module import name form of import statement.

From the surround module for example, you might use:
 
```python
from . import echo
from .. import formats
from ..filters import equalizer
```


## Decorators
The Python language provides a simple yet powerful syntax called ‘decorators’. A decorator is a function or a class that wraps (or decorates) a function or a method. The ‘decorated’ function or method will replace the original ‘undecorated’ function or method. **Because functions are first-class objects in Python, this can be done ‘manually’, but using the @decorator syntax is clearer and thus preferred.**

In [1]:
def foo():
    # do something
    print("I'm foo.")

def decorator(func):
    # manipulate func
    print("I'm {fname}, and I'm decorated.".format(fname=func.__name__ ))
    return func

In [2]:
foo()

I'm foo.


In [3]:
foo = decorator(foo)  # Manually decorate

I'm foo, and I'm decorated.


In [4]:
@decorator
def bar():
    # Do something
    print("I'm bar, this is my core logic.")
# bar() is decorated

bar()

I'm bar, and I'm decorated.
I'm bar, this is my core logic.


In [5]:
bar.__name__

'bar'

This mechanism is useful for separating concerns and avoiding external un-related logic ‘polluting’ the core logic of the function or method. 

A good example of a piece of functionality that is better handled with decoration is memoization or caching: you want to store the results of an expensive function in a table and use them directly instead of recomputing them when they have already been computed. This is clearly not part of the function logic.

## Context Manager

A context manager is a Python object that provides extra contextual information to an action.

This extra information takes the form of running a callable upon initiating the context using the `with` statement, as well as running a callable upon completing all the code inside the with block. 

### Example -- open a file
The most well known example of using a context manager is shown here, opening on a file:


In [6]:
with open('00-pythonic-overview.ipynb') as f:
    contents = f.read()

Anyone familiar with this pattern knows that invoking open in this fashion ensures that f‘s close method will be called at some point. This reduces a developer’s cognitive load and makes the code easier to read.

Let’s implement the above functionality ourselves, starting with the class approach:

In [7]:
class CustomOpen(object):
    def __init__(self, filename):
        self.file = open(filename)

    def __enter__(self):
        return self.file

    def __exit__(self, ctx_type, ctx_value, ctx_traceback):
        self.file.close()

with CustomOpen('00-pythonic-overview.ipynb') as f:
    contents = f.read()

And now the generator approach using Python’s own contextlib:

In [8]:
from contextlib import contextmanager

@contextmanager
def custom_open(filename):
    f = open(filename)
    try:
        yield f
    finally:
        f.close()

with custom_open('00-pythonic-overview.ipynb') as f:
    contents = f.read()

Since the two approaches appear the same, we should follow the Zen of Python to decide when to use which. The class approach might be better if there’s a considerable amount of logic to encapsulate. The function approach might be better for situations where we’re dealing with a simple action.

## Dynamic Typing

Python is dynamically typed, which means that variables do not have a fixed type. In fact, in Python, variables are very different from what they are in many other languages, specifically statically-typed languages.

Variables are not a segment of the computer’s memory where some value is written, they are ‘tags’ or ‘names’ pointing to objects.

Some guidelines help to avoid this issue:
- Avoid using the same variable name for different things.

**Bad**  
```python
a = 1
a = 'a string'
def a():
    pass  # Do something
```
**Good**  
```python
count = 1
msg = 'a string'
def func():
    pass  # Do something
```

- Using short functions or methods helps reduce the risk of using the same name for two unrelated things.

- It is better to use different names even for things that are related, when they have a different type:

**Bad**  
```python
items = 'a b c d'  # This is a string...
items = items.split(' ')  # ...becoming a list
items = set(items)  # ...and then a set
```

### Reassign

There is no efficiency gain when reusing names: the assignments will have to create new objects anyway.   
However, when the complexity grows and each assignment is separated by other lines of code, including ‘if’ branches and loops, it becomes harder to ascertain what a given variable’s type is.

Some coding practices, like functional programming, recommend never reassigning a variable. In Java this is done with the final keyword. Python does not have a final keyword and it would be against its philosophy anyway.   
However, it may be a good discipline to avoid assigning to a variable more than once, and it helps in grasping the concept of mutable and immutable types.


## Mutable and immutable types

Python has two kinds of built-in or user-defined types.

- Mutable types are those that allow in-place modification of the content. Typical mutables are lists and dictionaries: All lists have mutating methods, like list.append() or list.pop(), and can be modified in place. The same goes for dictionaries.

- Immutable types provide no method for changing their content. For instance, the variable x set to the integer 6 has no “increment” method. If you want to compute x + 1, you have to create another integer and give it a name.

Using properly mutable types for things that are mutable in nature and immutable types for things that are fixed in nature helps to clarify the intent of the code.For example, the immutable equivalent of a list is the tuple, created with (1, 2). This tuple is a pair that cannot be changed in-place, and can be used as a key for a dictionary.

One peculiarity of Python that can surprise beginners is that **strings are immutable**. This means that when constructing a string from its parts, it is much more efficient to accumulate the parts in a list, which is mutable, and then glue (‘join’) the parts together when the full string is needed. One thing to notice, however, is that list comprehensions are better and faster than constructing a list in a loop with calls to append().

One other option is using the map function, which can ‘map’ a function (‘str’) to an iterable (‘range(20)’). This results in a map object, which you can then (‘join’) together just like the other examples. The map function can be even faster than a list comprehension in some cases.

**Bad**  

In [9]:
%%timeit 
# create a concatenated string from 0 to 19 (e.g. "012..1819")
nums = ""
for n in range(2000):
    nums += str(n)   # slow and inefficient


532 µs ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Good**  

In [10]:
%%timeit 
# create a concatenated string from 0 to 19 (e.g. "012..1819")
nums = []
for n in range(2000):
    nums.append(str(n))
nums = "".join(nums)  # much more efficient

518 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Better**  

In [11]:
%%timeit
# create a concatenated string from 0 to 19 (e.g. "012..1819")
nums = [str(n) for n in range(2000)]
nums = "".join(nums)

421 µs ± 9.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Best**

In [12]:
%%timeit
# create a concatenated string from 0 to 19 (e.g. "012..1819")
nums = map(str, range(2000))
nums = "".join(nums)

316 µs ± 5.65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


One final thing to mention about strings is that using join() is not always best. 
- In the instances where you are creating a new string from a pre-determined number of strings, using the addition operator is actually faster.
- But in cases like above or in cases where you are adding to an existing string, using join() should be your preferred method.

In [13]:
foo = 'foo'
bar = 'bar'

foobar = foo + bar  # This is good
foo += 'ooo'  # This is bad, instead you should do:
foo = ''.join([foo, 'ooo'])

**Note**
```
You can also use the % formatting operator to concatenate a pre-determined number of strings besides str.join() and +. 

However, PEP 3101, discourages the usage of the % operator in favor of the str.format() method.
```

In [14]:
foo = 'foo'
bar = 'bar'

foobar = '%s%s' % (foo, bar) # It is OK
foobar = '{0}{1}'.format(foo, bar) # It is better
foobar = '{foo}{bar}'.format(foo=foo, bar=bar) # It is best

## Reference
- [Coding-Guide](https://github.com/ecmadao/Coding-Guide/blob/master/Notes/Python/Python%E8%BF%9B%E9%98%B6/Effective%20Python.md)